# 参数管理：详细讲解

---

## 一、准备工作：先搭一个简单网络

```python
import torch
from torch import nn

net = nn.Sequential(
    nn.Linear(4, 8),      # 第0层：全连接，4进8出
    nn.ReLU(),             # 第1层：激活函数（没有参数）
    nn.Linear(8, 1)        # 第2层：全连接，8进1出
)

X = torch.rand(2, 4)      # 2个样本，每个4个特征
output = net(X)            # 输出形状：2×1
```

**网络结构：**

```
net[0] = Linear(4, 8)     ← 有参数（weight + bias）
net[1] = ReLU()            ← 没有参数！
net[2] = Linear(8, 1)     ← 有参数（weight + bias）
```

---

## 二、怎么访问参数？

### 2.1 net就像一个列表

**Sequential就像一个列表，用编号取每一层：**

```python
net[0]    # 第0层：Linear(4, 8)
net[1]    # 第1层：ReLU
net[2]    # 第2层：Linear(8, 1)
```

**就像书架上的书：第0本、第1本、第2本。**

### 2.2 每个Linear层里有什么？

**每个全连接层里面有两样东西：**

```
weight（权重）：一个矩阵，用来做计算的
bias（偏置）：一个向量，加上去的偏移
```

**怎么看？**

```python
net[2].state_dict()
```

```
输出：
OrderedDict([
    ('weight', tensor([[-0.12, 0.34, ...]])),    # 权重矩阵
    ('bias', tensor([0.05]))      # 偏置
])
```

**翻译：**

```
net[2]           → 拿出第2层
.state_dict()    → 把这一层的所有参数列出来
```

**state_dict就是"状态字典"，把参数的名字和值配对列出来。**

### 2.3 直接访问某个具体参数

```python
net[2].weight        # 拿出第2层的权重（是一个Parameter对象）
net[2].bias          # 拿出第2层的偏置

net[2].weight.data   # 拿出权重的具体数值
net[2].bias.data     # 拿出偏置的具体数值

net[2].weight.grad   # 拿出权重的梯度（还没训练就是None）
```

**为什么有 .data 和 .grad？**

```
一个参数(Parameter)包含两样东西：
├── .data  → 参数本身的值（比如权重是多少）
└── .grad  → 这个参数的梯度（训练时反向传播算出来的）

还没训练的时候，.grad = None（因为还没算过梯度）
```

**比喻：**

```
一个学生(Parameter)有两个属性：
├── .data  → 他现在的成绩（比如85分）
└── .grad  → 他需要提高多少（比如+5分）

还没考试的时候，.grad = None（不知道该提高多少）
```

### 2.4 一次性看所有层的参数

```python
for name, param in net.named_parameters():
    print(name, param.shape)
```

```
输出：
0.weight    torch.Size([8, 4])     # 第0层权重：8×4的矩阵
0.bias      torch.Size([8])        # 第0层偏置：8个数
2.weight    torch.Size([1, 8])     # 第2层权重：1×8的矩阵
2.bias      torch.Size([1])        # 第2层偏置：1个数
```

**注意：第1层（ReLU）没有出现，因为ReLU没有参数！**

**名字的规则：**

```
"0.weight"  → 第0层的权重
"0.bias"    → 第0层的偏置
"2.weight"  → 第2层的权重
"2.bias"    → 第2层的偏置
```

---

## 三、怎么初始化参数？（重要！）

### 3.1 为什么要初始化？

**回忆Xavier初始化那一节：**

```
权重太大 → 信号爆炸
权重太小 → 信号消失
权重刚好 → 训练正常
```

**PyTorch有默认的初始化，但有时候你想自己控制。**

### 3.2 最常用的初始化方法

```python
def init_normal(m):
    if type(m) == nn.Linear:                    # 如果是全连接层
        nn.init.normal_(m.weight, mean=0, std=0.01)  # 权重：正态分布
        nn.init.zeros_(m.bias)                         # 偏置：全设为0

net.apply(init_normal)    # 对net里的每一层都执行这个函数
```

### 逐行详细解释

**第1行：定义一个函数**

```python
def init_normal(m):
```

```
定义了一个函数叫 init_normal
m 是传进来的"一个层"
（apply会自动把每一层依次传进来）
```

**第2行：判断这个层是不是Linear**

```python
if type(m) == nn.Linear:
```

```
为什么要判断？
因为网络里有各种层：Linear、ReLU、BatchNorm...
ReLU没有参数，你不能对它初始化
所以要先看看：这个层是Linear吗？
    是 → 执行初始化
    不是 → 跳过，什么都不做
```

**第3行：初始化权重**

```python
nn.init.normal_(m.weight, mean=0, std=0.01)
```

```
nn.init          → PyTorch提供的初始化工具包
.normal_         → 用正态分布填充（下划线表示"直接替换，不返回新值"）
m.weight         → 这一层的权重
mean=0           → 均值为0
std=0.01         → 标准差为0.01（很小的随机数）
```

**第4行：初始化偏置**

```python
nn.init.zeros_(m.bias)
```

```
把偏置全部设为0
这是最常见的做法
```

**第5行：apply**

```python
net.apply(init_normal)
```

```
apply的意思：对net里面的每一层，依次执行init_normal函数

它会这样做：
    init_normal(net[0])    → net[0]是Linear → 执行初始化 ✓
    init_normal(net[1])    → net[1]是ReLU   → 不是Linear → 跳过
    init_normal(net[2])    → net[2]是Linear → 执行初始化 ✓
```

**比喻：**

```
apply就像一个检查员，挨个房间检查：
    第0个房间：是仓库（Linear）→ 整理一下 ✓
    第1个房间：是厕所（ReLU） → 不用整理，跳过
    第2个房间：是仓库（Linear）→ 整理一下 ✓
```

### 3.3 对不同层用不同的初始化

```python
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)    # Xavier初始化

def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)       # 全部设为1（别这么干！纯演示）

net[0].apply(init_xavier)      # 第0层用Xavier
net[2].apply(init_constant)    # 第2层用常数
```

```
因为每一层本身也是Module
所以可以单独对某一层调用apply
不用整个网络一起改
```

---

## 四、怎么冻结参数？（微调必用！）

### 4.1 什么叫冻结？

```
正常训练：所有参数都会被更新
冻结某层：这一层的参数不动，只训练其他层
```

**什么时候用？**

```
微调(Fine-tuning)的时候！

比如你用了一个别人训练好的大模型
你只想改最后一层，前面的层保持不变
→ 冻结前面的层，只训练最后一层
```

### 4.2 怎么冻结？

**核心：设置 requires_grad = False**

```python
# 冻结第0层的所有参数
for param in net[0].parameters():
    param.requires_grad = False
```

**翻译：**

```
net[0].parameters()      → 拿出第0层的所有参数
param.requires_grad      → 这个参数需要计算梯度吗？
= False                  → 不需要！不要动它！
```

**设成False之后：**

```
训练的时候：
    net[0]的权重和偏置 → 不会被更新（冻住了）❄️
    net[2]的权重和偏置 → 正常更新 ✓
```

### 4.3 微调的完整例子

```python
# 假设net是别人训练好的模型

# 第1步：冻结所有层
for param in net.parameters():
    param.requires_grad = False

# 第2步：只解冻最后一层
for param in net[2].parameters():
    param.requires_grad = True

# 现在训练的话，只有最后一层会更新
```

**比喻：**

```
一栋楼有3层
你搬进来，发现1楼和2楼装修得很好，不想动
只想重新装修3楼

第1步：把所有楼层都锁上（全部冻结）
第2步：只打开3楼的门（解冻最后一层）
第3步：开始装修（训练）→ 只有3楼会变
```

### 4.4 检查哪些参数被冻结了

```python
for name, param in net.named_parameters():
    print(name, param.requires_grad)
```

```
输出：
0.weight    False     ← 冻住了
0.bias      False     ← 冻住了
2.weight    True      ← 可以训练
2.bias      True      ← 可以训练
```

---

## 六、总结

### 必须记住

```python
# 1. 访问参数
net[2].weight.data        # 看某一层的权重值
net[2].bias.data          # 看某一层的偏置值

# 2. 初始化参数
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)

# 3. 冻结参数（微调必用！）
for param in net[0].parameters():
    param.requires_grad = False    # 这一层不训练了
```

### 核心概念

| 操作 | 代码 | 什么时候用 |
|:---|:---|:---|
| 看参数 | `net[2].weight.data` | 调试、检查 |
| 初始化 | `nn.init.normal_()` + `apply` | 训练开始前 |
| 冻结 | `requires_grad = False` | **微调时！** |

### 一句话总结

> 参数管理 = **看参数**（访问）+ **改参数**（初始化）+ **锁参数**（冻结）